In [0]:
%sql
use catalog `get_started`;
use schema `labuser`;


-- create a temp view
CREATE OR REPLACE TEMP VIEW temp_view_employees_silver AS
SELECT ID, Firstname, Country, upper(Role) as Role, current_timestamp() as TimeStamp, date(timestamp) as Date 
FROM current_employees_bronze;

-- View the results of the temp view
SELECT * FROM temp_view_employees_silver; 

In [0]:
%sql
-- drop table if exists current_employees
DROP TABLE IF EXISTS current_employees_silver;

-- create an empty delta table and columns
CREATE TABLE IF NOT EXISTS current_employees_silver (
    ID INT,
    Firstname STRING,
    Country STRING,
    Role STRING,
    TimeStamp TIMESTAMP,
    Date DATE
);
    
-- Insert records from the view when not matched with the target silver table
MERGE INTO current_employees_silver AS target
USING temp_view_employees_silver AS source
-- ON current_employees_silver.ID = temp_view_employees_silver.ID
ON target.ID = source.ID
WHEN NOT MATCHED THEN INSERT *;  -- insert records if not match


-- View bronze table
SELECT * FROM current_employees_silver;

In [0]:
%sql
-- create a temp view
CREATE OR REPLACE TEMP VIEW temp_view_total_roles AS
SELECT Role,count(*) as TotalEmployees
FROM current_employees_silver
GROUP BY Role;

-- View the results of the temp view
SELECT * FROM temp_view_total_roles; 

In [0]:
%sql
-- drop table if exists current_employees
DROP TABLE IF EXISTS total_roles_gold;

-- create an empty delta table and columns
CREATE TABLE IF NOT EXISTS total_roles_gold (
    Role STRING,
    TotalEmployees INT
);

In [0]:
%sql
-- Insert overwrite to give it history rather than dropping it
INSERT OVERWRITE TABLE total_roles_gold
SELECT * FROM temp_view_total_roles;

In [0]:
%sql
SELECT * FROM total_roles_gold